In [1]:
import pandas as pd
import scipy.signal as signal
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats as st
import glob
import sklearn
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from datetime import date
import sklearn.preprocessing as prepro
import sklearn.cluster as cluster
import sklearn.linear_model as lm
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

This notebook works with pandas version 1.2.4 and sklearn version 0.24.2 Please check your requirements. Place ooportunity dataset files in a data folder. The header.csv file with this code contains the names of columns of the files

In [2]:
pd.__version__

'1.2.4'

In [3]:
sklearn.__version__

'0.24.2'

# Dataset information
This code uses the opportunity dataset that is available in the UCI ML repository. Please download from https://archive.ics.uci.edu/ml/datasets/OPPORTUNITY+Activity+Recognition and use in combination with the header.csv file provided with this code. 

# Configuration options

In [4]:
pd.set_option('max_columns', None)
save_folder = 'data' # use to save results
window_size = 30
step = 15

In [ ]:
#Label columns: 0: Locomotion, 1: HL_Activity, 2: LL_Left_Arm, 3: LL_Left_Arm_Object
# 4: LL_Right_Arm, 5:LL_Right_Arm_Object, 6: ML_Both_Arms
label_cols={'HL':1, 'LOC':0, 'LL_LeftArm':2, "LL_LArm_Obj":3, "LL_RArm":4, "LL_RArm_Obj":5, "ML_BArms":6}

# Function definitions

In [6]:
def read_opp_file(file):
    header_path = 'header.csv'
    header=pd.read_csv(header_path, sep='|', names=['column'])
    header = header.column.str.replace(",","")
    data = pd.read_csv(file, sep=' ', header=None, names=header)
    nan_ix = pd.isnull(data).sum(axis=1)>data.shape[1]*0.60 #more than 60% of columns are null in the row
    data = data[~nan_ix]
    data.loc[:,'MILLISEC']=pd.to_datetime(data.MILLISEC, unit='ms')
    return data

In [7]:
def make_frame(data, user="user", use_acc="", label_column=0):
    labels = data.iloc[:,243:]
    labels = labels.iloc[:,label_column].astype('category')
    data=data.iloc[:, :243]
    columns_keep = list(filter(lambda x: ('acc' in x or 'gyro' in x or 'magnetic' in x) and use_acc in x, data.columns))
    time = data["MILLISEC"]
    data = data[columns_keep]
    #nan_ix = pd.isnull(data).sum(axis=1)>data.shape[1]*0.90 #more than 90% of columns are null in the row
    #data = data[~nan_ix]
    #time = time[~nan_ix]
    #labels = labels[~nan_ix]
    indexes = data.index
    
    #fill na
    c = signal.savgol_filter(data, window_length=3, polyorder=2, axis=0, deriv=0)  
    imputer = SimpleImputer(missing_values=np.nan, strategy='median')
    c = imputer.fit_transform(c)
    data = pd.DataFrame.from_records(c, columns=columns_keep, index=indexes)
    
    data['user'] = user
    data['label'] = labels
    data['MILLISEC'] = time
    return data

In [8]:
def fill_nulls(df):
    df = df.fillna(method="bfill")
    df = df.fillna(method="ffill")
    return df

In [9]:
def make_features(data, window_size=30, step=15):
    '''
    window size and step are in seconds
    '''
    data.index=data["MILLISEC"]
    
    labels = data['label']
    user = data['user'].unique()[0]
    columns = data.columns[~data.columns.isin(['user', 'label', 'MILLISEC'])]
    
    keep = data["MILLISEC"].dt.second % step 
    keep = keep - keep.shift() < 0
    
    wt = str(int(window_size))+"S"
    means = data[columns].rolling(wt).mean()
    means = fill_nulls(means)[keep]
    means.columns = [str(col) + '_mean' for col in means.columns]
    
    variances = data[columns].rolling(wt).std()
    variances = fill_nulls(variances)[keep]
    variances.columns = [str(col) + '_var' for col in variances.columns]
    
    minimum = data[columns].rolling(wt).min()
    minimum = fill_nulls(minimum)[keep]
    maximum = data[columns].rolling(wt).max()
    maximum = fill_nulls(maximum)[keep]
    ranged = maximum - minimum
    ranged.columns = [str(col) + '_range' for col in minimum.columns]
    
    medians = data[columns].rolling(wt).median()
    medians = fill_nulls(medians)[keep]
    medians = pd.DataFrame(medians.values, medians.index, columns=medians.columns)
    medians.columns = [str(col) + '_median' for col in medians.columns] 
    
    labels.index = data.index
    mode_labels = labels.rolling(wt).apply(lambda x: st.mode(x)[0])[keep]
    features = pd.concat([means, variances, ranged, medians, mode_labels], axis=1)
    features['user']=user
    return features

# Read files

## Read Additional data files

Additional data consists of sensors not available in real conditions. Usually more sensors and of higher quality. We use InertialMeasurementUnit sensors as additional data

In [10]:
path = 'data/*-Drill.dat'
files = glob.glob(path)
drill_data = pd.DataFrame()
use_acc = 'InertialMeasurementUnit'

for file in files:
    user = int(file[-11])
    data = read_opp_file(file)
    data = make_frame(data, use_acc=use_acc, user=user, label_column=1)
    print("Read file"+file)
    features = make_features(data, window_size=window_size, step=step)
    drill_data = pd.concat([drill_data, features])
    
path = 'data/*-ADL*.dat'
files = glob.glob(path)
for file in files:
    user = int(file[-10])
    data = read_opp_file(file)
    print("Read file"+file)
    data = make_frame(data, use_acc=use_acc, user=user, label_column=1)
    features = make_features(data, window_size=window_size, step=step)
    drill_data = pd.concat([drill_data, features])

Read filedata\S1-Drill.dat
Read filedata\S2-Drill.dat
Read filedata\S3-Drill.dat
Read filedata\S4-Drill.dat
Read filedata\S1-ADL1.dat
Read filedata\S1-ADL2.dat
Read filedata\S1-ADL3.dat
Read filedata\S1-ADL4.dat
Read filedata\S1-ADL5.dat
Read filedata\S2-ADL1.dat
Read filedata\S2-ADL2.dat
Read filedata\S2-ADL3.dat
Read filedata\S2-ADL4.dat
Read filedata\S2-ADL5.dat
Read filedata\S3-ADL1.dat
Read filedata\S3-ADL2.dat
Read filedata\S3-ADL3.dat
Read filedata\S3-ADL4.dat
Read filedata\S3-ADL5.dat
Read filedata\S4-ADL1.dat
Read filedata\S4-ADL2.dat
Read filedata\S4-ADL3.dat
Read filedata\S4-ADL4.dat
Read filedata\S4-ADL5.dat


## Read Test data 
Test data is the data from real-life sensors. We use accelerometers. We use drill data only to learn the mapping functions from the real-life sensor to the data-space learned from additional data

In [11]:
all_data = pd.DataFrame()
#ADL Data
path = 'data/*-ADL*.dat'
files = glob.glob(path)
use_acc = 'BodyAccelerometer'
for file in files:
    user = int(file[-10])
    data = read_opp_file(file)
    print("Read file"+file)
    data = make_frame(data, use_acc=use_acc, user=user, label_column=1)
    features = make_features(data, window_size=window_size, step=step)
    all_data = pd.concat([all_data, features])
    
path = 'data/*-Drill.dat'
files = glob.glob(path)
mapping_data = pd.DataFrame()
for file in files:
    user = int(file[-11])
    data = read_opp_file(file)
    data = make_frame(data, use_acc=use_acc, user=user, label_column=1)
    print("Read file"+file)
    features = make_features(data, window_size=window_size, step=step)
    mapping_data = pd.concat([mapping_data, features])
mapping_data = pd.concat([mapping_data, all_data])

Read filedata\S1-ADL1.dat
Read filedata\S1-ADL2.dat
Read filedata\S1-ADL3.dat
Read filedata\S1-ADL4.dat
Read filedata\S1-ADL5.dat
Read filedata\S2-ADL1.dat
Read filedata\S2-ADL2.dat
Read filedata\S2-ADL3.dat
Read filedata\S2-ADL4.dat
Read filedata\S2-ADL5.dat
Read filedata\S3-ADL1.dat
Read filedata\S3-ADL2.dat
Read filedata\S3-ADL3.dat
Read filedata\S3-ADL4.dat
Read filedata\S3-ADL5.dat
Read filedata\S4-ADL1.dat
Read filedata\S4-ADL2.dat
Read filedata\S4-ADL3.dat
Read filedata\S4-ADL4.dat
Read filedata\S4-ADL5.dat
Read filedata\S1-Drill.dat
Read filedata\S2-Drill.dat
Read filedata\S3-Drill.dat
Read filedata\S4-Drill.dat


# Train functions definition

In [12]:
def traditional_validation(X, y, users):
    logo = LeaveOneGroupOut()
    all_y_test = []
    all_y_pred =[]
    for train_index, test_index in logo.split(X, y, users):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf = SVC(gamma='scale', class_weight='balanced').fit(X_train, y_train)
        pred = clf.predict(X_test)
        all_y_test.extend(y_test)
        all_y_pred.extend(pred)
    return all_y_test, all_y_pred

In [95]:
def fit_mapping_functions(tr_features, one_sensors_features, mapping_function, column, logistic): 
    z_train = np.copy(tr_features[:,column]).reshape(-1,1)
    
    # normarization function           
    if logistic:
        binarizer = prepro.Binarizer(threshold = np.median(z_train))
        z_train = binarizer.fit_transform(z_train).ravel()
        
    mapping_function.fit(one_sensors_features, z_train) # learning
    "test mapping function"
    score = mapping_function.score(one_sensors_features, z_train)
    return mapping_function, score

In [97]:
def train_test_proposed(one_acc_features, tr_features, mapping_features, labels_, users, mapping_function, logistic):
    #train combined model
    
    the_labels_ = labels_
    
    logo = LeaveOneGroupOut()
    y_true = []
    y_pred = []
        
    for train_index, test_index in logo.split(one_acc_features, the_labels_, users):
        y_train, y_test = the_labels_[train_index], the_labels_[test_index]
        x_train, x_test = np.copy(one_acc_features[train_index]), np.copy(one_acc_features[test_index])

        
        test_columns = np.empty((len(x_test),0))
        train_columns = np.empty((len(x_train),0))

        f_clf = SVC(gamma='scale', class_weight='balanced',probability =True)

        scaler_x = prepro.StandardScaler()
        x_train = scaler_x.fit_transform(x_train)
        
        scores = []
        for column in range(tr_features.shape[1]):  #one classifier per column
            mapping_function, score = fit_mapping_functions(tr_features, mapping_features, mapping_function, column, logistic)# learning
            scores.append(score)
            # evaluate train data
            if logistic:
                pred_col_t = mapping_function.predict_proba(x_train)[:,1].reshape(-1,1)
            else:
                pred_col_t = mapping_function.predict(x_train).reshape(-1,1)
                
            train_columns = np.hstack((train_columns, pred_col_t))

            #evaluate test data
            x_test = scaler_x.fit_transform(x_test)#fit and transform
            if logistic:
                pred_col = mapping_function.predict_proba(x_test)[:,1].reshape(-1,1)
            else:
                pred_col = mapping_function.predict(x_test).reshape(-1,1)

            test_columns = np.hstack((test_columns, pred_col))
        
        
        f_clf.fit(train_columns, y_train)#better to use train_columns as they are predicted
        pred = f_clf.predict(test_columns)

        y_true.extend(y_test)
        y_pred.extend(pred)
    return(y_true, y_pred, test_columns, np.mean(scores))

In [98]:
def train_test_proposed_boosted(one_acc_features, tr_features, mapping_features, labels_, users, mapping_function, logistic):
    #train combined model
    le = prepro.LabelEncoder()
    the_labels_ = le.fit_transform(labels_)
    
    logo = LeaveOneGroupOut()
    y_true = []
    y_pred = []
    
    for train_index, test_index in logo.split(one_acc_features, the_labels_, users):
        y_train, y_test = the_labels_[train_index], the_labels_[test_index]
        x_train, x_test = np.copy(one_acc_features[train_index]), np.copy(one_acc_features[test_index])
        
        #one classifier per column
        test_columns = np.empty((len(x_test),0))
        train_columns = np.empty((len(x_train),0))

        f_clf = SVC(gamma='scale', class_weight='balanced', probability=True)

        scaler_x = prepro.StandardScaler()
        x_train = scaler_x.fit_transform(x_train)
        
        scores = []

        for column in range(tr_features.shape[1]):
            mapping_function, score = fit_mapping_functions(tr_features, mapping_features, mapping_function, column, logistic)# learning
            scores.append(score)
            #evaluate train data
            if logistic:
                pred_col_t = mapping_function.predict_proba(x_train)[:,1].reshape(-1,1)
            else:
                pred_col_t = mapping_function.predict(x_train).reshape(-1,1)
            train_columns = np.hstack((train_columns, pred_col_t))

            #evaluate test data
            x_test = scaler_x.fit_transform(x_test)
            if logistic:
                pred_col = mapping_function.predict_proba(x_test)[:,1].reshape(-1,1)
            else:
                pred_col = mapping_function.predict(x_test).reshape(-1,1)

            test_columns = np.hstack((test_columns, pred_col))

        k = len(np.unique(labels_))
        w_1 = np.full([len(x_train), ], 1/k)
        

        f_clf.fit(train_columns, y_train, sample_weight= w_1)
            
       
        predicted = f_clf.predict(train_columns)
        errors_cluster_model = predicted != y_train
        
        err_1 = np.mean(np.average(errors_cluster_model, axis=0, weights=w_1))
        alpha_1 = np.log((1-err_1)/err_1)+ np.log (k-1)
        w_2 = w_1 * np.exp(alpha_1*errors_cluster_model)
        
        
        estimators = []
        estimators.append(('standardize', prepro.StandardScaler()))
        estimators.append(('clf', SVC(gamma='scale', probability=True, class_weight='balanced')))
        acc_model = Pipeline(estimators)
        
        acc_model.fit(x_train, y_train, **{'clf__sample_weight': w_2})
        
        predicted_acc = acc_model.predict(x_train)
        errors_acc = predicted_acc != y_train
        
        err_2 = np.mean(np.average(errors_acc, axis=0, weights=w_2))
        alpha_2 = np.log((1-err_2)/err_2)+ np.log (k-1) ### 
        
        
        t2_x = acc_model.predict_proba(x_test)
        t1_x = f_clf.predict_proba(test_columns)
        #get arg max
        sum_output = alpha_1*t1_x + alpha_2*t2_x
        pred = np.argmax(sum_output, axis=1)
    
        y_true.extend(y_test.astype(int))
        y_pred.extend(pred)
        
    return(y_true, y_pred, np.mean(scores) )

In [99]:
def make_proposed_features(X_add, n_clusters=None):
    scale = prepro.StandardScaler()
    X_feat = scale.fit_transform(X_add)
    cluster_model= None
    if n_clusters is None:
        dist = euclidean_distances(X_feat)
        d_threshold = np.std(dist)
        cluster_model = cluster.FeatureAgglomeration(distance_threshold=d_threshold, n_clusters=None, compute_full_tree=True)
    else:
        cluster_model = cluster.FeatureAgglomeration(distance_threshold=None, n_clusters=n_clusters, compute_full_tree=None)
    # normalization
    tr_features = cluster_model.fit_transform(X_feat)
    return scale, tr_features

# Train traditional approach

In [100]:
trial = 11
test_sensors = ['HIP', 'BACK', 'LUA_', 'RUA_', 'LUA^', 'RUA^', 'LWR', 'RWR']

In [48]:
save_file = repr(str(date.today()))+'_'+str(trial)+'_traditional_results.txt'

In [19]:
results_file=open(save_folder+save_file, "wt")
print(date.today(), file=results_file)
print("-----------", file=results_file)
print(f"Using {window_size} and {step}", file=results_file)

#Train with all sensors
users = all_data['user'].values
Y = all_data['label'].values
X = all_data.drop(['user', 'label'], axis=1).values
y_test, y_pred = traditional_validation(X, Y, users)
report = classification_report(y_test, y_pred, zero_division=0)
print(f'Traditional ALL Report')
print(report)
print (f'Traditional ALLReport \r\n {report}', file=results_file)
print("-----------END-----------", file=results_file)

for sensor in test_sensors:
    columns = list(filter(lambda x: sensor in x,all_data.columns))
    users = all_data['user'].values
    Y = all_data['label'].values
    X = all_data[columns].values
    y_test, y_pred = traditional_validation(X, Y, users)
    report = classification_report(y_test, y_pred, zero_division=0)
    print(f'Traditional {sensor} Report')
    print(report)
    print (f'Traditional {sensor} Report \r\n {report}', file=results_file)
    print("-----------END-----------", file=results_file)
results_file.close()


Traditional ALL Report
              precision    recall  f1-score   support

         0.0       0.78      0.32      0.45       215
       101.0       0.66      0.62      0.64       103
       102.0       0.41      0.64      0.50       195
       103.0       0.75      0.66      0.70       310
       104.0       0.36      0.80      0.50       190
       105.0       0.86      0.50      0.63       394

    accuracy                           0.58      1407
   macro avg       0.64      0.59      0.57      1407
weighted avg       0.68      0.58      0.59      1407

Traditional HIP Report
              precision    recall  f1-score   support

         0.0       0.33      0.24      0.27       215
       101.0       0.78      0.67      0.72       103
       102.0       0.33      0.48      0.39       195
       103.0       0.49      0.58      0.53       310
       104.0       0.33      0.54      0.41       190
       105.0       0.61      0.31      0.41       394

    accuracy                   

# Train proposed approach

In [105]:
save_file = repr(str(date.today()))+'_'+str(trial)+'_proposed_results.txt'
results_file=open(save_folder+save_file, "wt")
print(date.today(), file=results_file)
print("-----------", file=results_file)
print(f"Using {window_size} and {step}", file=results_file)

In [106]:
 #make cluster features
all_features = drill_data.drop(['user', 'label'], axis=1).reset_index(drop=True)
all_features = pd.concat([all_features,mapping_data.drop(['user', 'label'], axis=1).reset_index(drop=True)], axis=1, ignore_index=True)
all_features.fillna(0, inplace=True)

In [107]:
scale, tr_features  = make_proposed_features(all_features.values, n_clusters=30) #using 30
print(f"Using {tr_features.shape[1]} clusters", file=results_file)
print("-----------", file=results_file)

In [108]:
tr_features.shape

(1901, 30)

In [109]:
import warnings
warnings.filterwarnings('ignore')

In [111]:
test_sensors = ['RUA^', 'LUA^', 'HIP',  'LWR', 'RWR', 'RUA_', 'LUA_', 'BACK']
for sensor in test_sensors:
    columns = list(filter(lambda x: sensor in x,all_data.columns))
    users = all_data['user'].values
    Y = all_data['label'].values
    one_sensor_X= all_data[columns].values
    
    print(f"using{sensor}")
    mapping_features = mapping_data[columns] #using data from drill and adl files but same sensor as test sensor. 
    scale = prepro.StandardScaler()
    mapping_features = scale.fit_transform(mapping_features)
    
    #train proposed
    #mapping_function = lm.LogisticRegression(solver="lbfgs", max_iter=100000) #for logistic regression
    mapping_function = SVR(kernel="linear", C=1.0, epsilon=0.00001) #for linear regression, epsilon is small as values of features are small
    y_true, y_pred, mean_score = train_test_proposed_boosted(one_sensor_X,  tr_features, mapping_features, Y, users, mapping_function, False)
    print(mean_score)
    print(f"Proposed Boosted {sensor}")
    report = classification_report(y_true, y_pred, zero_division=0)
    print(report)
    print(f'Proposed Boosted {sensor} Report \r\n {report}', file=results_file)
    
    #mapping_function = lm.LogisticRegression(solver="lbfgs", max_iter=100000)#for logistic regression
    y_true, y_pred, test_col, mean_score = train_test_proposed(one_sensor_X, tr_features, mapping_features, Y, users, mapping_function, False)
    print(mean_score)
    print(f"Proposed  {sensor}")
    report = classification_report(y_true, y_pred, zero_division=0)
    print(report)
    print(f'Proposed  {sensor} Report \r\n {report}', file=results_file)
    
    
results_file.close()
    

usingRUA^
0.3453761155034671
Proposed Boosted RUA^
              precision    recall  f1-score   support

           0       0.33      0.34      0.34       215
           1       0.69      0.64      0.66       103
           2       0.45      0.15      0.22       195
           3       0.55      0.65      0.60       310
           4       0.42      0.30      0.35       190
           5       0.52      0.69      0.59       394

    accuracy                           0.50      1407
   macro avg       0.49      0.46      0.46      1407
weighted avg       0.49      0.50      0.48      1407

0.3453761155034671
Proposed  RUA^
              precision    recall  f1-score   support

         0.0       0.45      0.45      0.45       215
       101.0       0.70      0.68      0.69       103
       102.0       0.33      0.39      0.36       195
       103.0       0.78      0.58      0.66       310
       104.0       0.39      0.74      0.51       190
       105.0       0.70      0.48      0.57    

In [ ]:
l_all = drill_data['label']
u_all = drill_data['user']

In [ ]:
y_test, y_pred = traditional_validation(tr_features, l_all, u_all)
report = classification_report(y_test, y_pred, zero_division=0)
print(f'tr_features ALL Report')
print(report)